# A closed-form solution to the Michaelis-Menten equation

Leonor Michaelis and Maud Menten proposed at the beginning of the XXth century a widely-used [enzyme kinetics](https://en.wikipedia.org/wiki/Michaelis%E2%80%93Menten_kinetics) to model the flux through a monosubstrate, single-step enzyme catalyzed reaction. enzyme kinetics for a monosubstrate, single-step reaction:

$$S + E \rightleftharpoons ES \rightarrow E + P$$

Under the quasi steady state assumption, _i.e._ when $d/dt(ES) = 0$, we arrive at the equation

$$v = \frac{v_{max} S}{K_M + S}$$

with

$$K_M = \frac{k_{-1} + k_{cat}}{k_1}$$

and

$$v_{max} = E k_{cat}.$$

Although, this equation is usually numerically integrated, a closed-form solution was proposed by <cite data-cite="6657738/T9S4TIAN"></cite>:

$$S(t) = K_M W \left( \frac{S_0}{K_M} \exp{\left( \frac{S_0 - v_{max}t}{K_M} \right)} \right),$$

where $W$ is [Lambert's W](https://en.wikipedia.org/wiki/Lambert_W_function) function. Equation (3), together with the identity:

$$ES(t) = \frac{E_0 S}{K_M + S} (1 - \exp{(-(K_M + S)k_1 t)})$$

and the conservation law

$$E(t) + ES(t) = E_0$$

allows us to obtain the solution to the dynamics of all the species in the chemical reaction. This closed-form solution should be used under conditions in agreement with the quasy steady state assumption, _i.e._ 

$$\frac{E_0}{K_M + S_0} << 1$$.

We will use the [lambert W](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.special.lambertw.html)
function from the scipy library to solve equation (3).

In [1]:
from scipy.special import lambertw
from matplotlib import pyplot as plt
from ipywidgets import interact, widgets
%matplotlib inline
# Let's define our W function which outputs the first branch of Lambert W
# and also only real part since the imaginery is always 0 for positive argument (the case)
def W(z):
    return np.real(lambertw(z, k=0))

# Let's define some constants and parameters
S_0 = 15 # mM
E_0 = 0.5 # mM
K_cat = 30 # s^-1
v_max = K_cat * E_0
k_1 = 5.5 # s^-1
k_minus_1 = 0.01 # s^-1
K_M = (k_minus_1 + K_cat) / k_1
t_0, t_f, dt = 0, 5, 0.01

def plot_func(K_M, K_cat, S_0, E_0, t_f):
    # Here are the functions for all species as well as the flux
    v_max = K_cat * E_0
    S = lambda t : K_M * W(S_0 / K_M * np.exp(1 / K_M * (-v_max * t + S_0)))
    ES = lambda t, S : (E_0 * S(t)) / (K_M + S(t)) * (1 - np.exp(-(K_M + S(t)) * k_1 * t))
    E = lambda t, ES : E_0 - ES(t, S)
    P = lambda t, S, ES : S_0 - S(t) - ES(t, S) 
    v = lambda t, S : (v_max * S(t)) / (K_M + S(t)) 
    T = np.arange(t_0, t_f, dt)

    # Plot the figures!
    plt.figure(figsize=(10, 12))
    plt.subplot(211)
    plt.plot(T, [P(t, S, ES) for t in T], label='P')
    plt.plot(T, [S(t) for t in T], label='S')
    plt.plot(T, [10/E_0 * E(t, ES) for t in T], label='E')
    plt.plot(T, [10/E_0 * ES(t, S) for t in T], label='ES')
    plt.ylabel('[X] (mM)', fontsize=18)
    plt.xlabel('t (s)', fontsize=18)
    plt.legend(loc='lower right')
    
    plt.subplot(212)
    plt.plot(T, [100 * v(t, S) / v_max for t in T])
    plt.ylabel('$f_{ES}$ (%)', fontsize=18)
    plt.xlabel('t (s)', fontsize=18)

interact(plot_func, 
         K_M=widgets.FloatSlider(min=0.1, max=50, step=0.1, value=0.7, description='$K_M$'),
         K_cat=widgets.FloatSlider(min=0, max=50, step=1, value=5, description='$K_{cat}$'),
         S_0=widgets.FloatSlider(min=0, max=50, step=1, value=15, description='$S_0$'),
         E_0=widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.1, description='$E_0$'),
         t_f=widgets.FloatSlider(min=1, max=50, step=1, value=50, description='$t_f$')
        )
plt.show()

interactive(children=(FloatSlider(value=0.7, description='$K_M$', max=50.0, min=0.1), FloatSlider(value=5.0, d…

With the current parameter values we have

$\frac{E_0}{K_M + S_0} =$ {{round(E_0/(K_M + E_0), 3)}}. Also note that starts with a velocity which is {{round(v(0, S) / v_max, 3)}}% of the $v_{max}$.

## Sensitivity of fluxes to fluctuating substrate concentrations
Enzymes get saturated by their substrates, meaning that all active sites of the enzyme can be occupied at the same time. In this scenario the enzyme is operating at full capacity, and hence, any further increase in substrate concentration won't affect the reaction flux. In Michaelis-Menten kinetics, an enzyme is saturated when the concentration of all its substrates is well above the $K_M$ value for each one of them. Now, previous studies have observed that many substrate concentrations in _E. coli_ are above their $K_M$s. Could it be that metabolic networks have evolved to sustain a saturated state for their enzymes? In this manner, the system would be practically insensitive to small fluctuations in metabolite concentrations, which poses an advantage from the control point of view. Since fluxes can then be regulated by controlling enzyme activity alone. As an example, we will plot the same system from above, but this time we explore how the flux varies with varying substrate concentrations which are below and well above the $K_M$ value.

In [2]:
def plot_func(K_M, S_mean, dE):
    v_max = 30
    t_f = 10
    T = np.arange(0, t_f, 0.1)
    max_e = dE * S_mean
    S = S_mean + (max_e - -max_e) * np.random.rand(len(T)) + (-max_e)
    v = lambda t, S : (v_max * S[t]) / (K_M + S[t]) 

    # Plot the figures!
    plt.figure(figsize=(14, 12))
    plt.subplot(211)
    plt.plot(T, [S[i] for i in range(len(T))], label='S')
    plt.hlines(K_M, xmin=0, xmax=t_f, colors='black', label='$K_M$')
    plt.ylabel('[X] (mM)', fontsize=18)
    plt.xlabel('t (s)', fontsize=18)
    plt.legend(loc='lower right')
    
    plt.subplot(212)
    plt.plot(T, [v(i, S) for i in range(len(T))], color='orange', label='v')
    plt.ylabel('v ($\mathrm{mM.s}^{-1}$)', fontsize=18)
    plt.xlabel('t (s)', fontsize=18)

interact(plot_func,
         K_M=widgets.FloatSlider(min=0.1, max=50, step=0.1, value=0.7, description='$K_M$'),
         S_mean=widgets.FloatSlider(min=0.1, max=50, step=0.1, value=10, description='$mean(S)$'),
         dE=widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.1, description='$dE$')
        )
plt.show()

interactive(children=(FloatSlider(value=0.7, description='$K_M$', max=50.0, min=0.1), FloatSlider(value=10.0, …

In [16]:
def plot_func(K_M, dE):
    v_max = 30
    t_f = 10
    T = np.arange(0, t_f, 0.1)
    S_means = np.linspace(0.1, 10, 100)
    v = lambda t, S : (v_max * S[t]) / (K_M + S[t])

    max_Amplitude = []
    for S_mean in S_means:
        max_e = dE * S_mean
        S = S_mean + (max_e - -max_e) * np.random.rand(len(T)) + (-max_e)
        V = [v(i, S) for i in range(len(T))]
        max_Amplitude.append(max(V) - min(V))

    # Plot the figures!
    plt.figure(figsize=(14, 8))
    plt.plot(S_means, max_Amplitude, color='black')
    plt.vlines(K_M, ymin=min(max_Amplitude), ymax=max(max_Amplitude), 
               colors='blue', linestyle="--")
    plt.ylabel('$A_{max}$', fontsize=18)
    plt.xlabel('mean [x]', fontsize=18)
    plt.text(K_M, 0.2, '$K_M$', fontsize=18, color='blue')

interact(plot_func,
         K_M=widgets.FloatSlider(min=0.1, max=50, step=0.1, value=0.7, description='$K_M$'),
         dE=widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.1, description='$dE$')
        )
plt.show()

interactive(children=(FloatSlider(value=0.7, description='$K_M$', max=50.0, min=0.1), FloatSlider(value=0.1, d…

## References
<div class="cite2c-biblio"></div>